<a href="https://colab.research.google.com/github/7ZXU/AI/blob/main/kaggle/IcebergClassifierChallenge/CV01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 빙하와 배를 구분해내는 모델 세우는 문제
- image classification
- CNN



런타임 > 런타임유형 > GPU 속도 빠름

In [ ]:
# 라이브러리 (=모듈)
#특정 기능을 담은 프로그램
#함수 or 클래스 포함

#패키지
#라이브러리 집합

import numpy as np
import pandas as pd

#numpy
#수치 계산을 위해서 만들어진 라이브러리 (데이터 분석에 사용)

#pandas
#데이터 다루기 위한 라이브러리

from sklearn.model_selection import train_test_split

#Scikit-Learn
#파이썬 머신러닝 라이브러리 

#sklearn.model_selection
#mode_selection 패키지

#train_test_split
#train set 과 test set 분리 모듈

import os
#os.path
#파일명과 파일경로에 대한 함수 제공

#join
#os.path.join(경로1, 경로2, 경로3, ...)
#여러 경로 -> 하나의 경로로 이어줌

from matplotlib import pyplot as plt
#matplotlib
#그래프 그릴 때 사용되는 라이브러리

from mpl_toolkits.mplot3d import Axes3D
#3차원 그래프 그리는 데에 사용되는 모듈

import pylab
#Matplotlib
#pyplot & pylab 모듈 포함

##############질문################
#왜 둘 다 import??
#두 라이브러리의 함수 따로 사용

#pyplot
#사용자 인터페이스 제공
#사용자가 보이지 않는 곳에서 명령을 받아 그래프 작성한 matplotlib
#사용자의 명령에 따라 자동으로 그래프 그리는 pyplot

#pylab
#pyplot & numpy 하나의 namespace에 import

plt.rcParams['figure.figsize']=10, 10
#그림크기 (가로, 세로)


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#mount : 드라이브 접근 허용


Mounted at /content/drive


In [ ]:
data_path = os.path.dirname('/content/drive/MyDrive/colab/Statoil/')


#경로복사

target = is_iceberg

iceberg == 1 (빙하)
iceberg == 0 (배)


HH : 수평 필터로 걸러냄
HV : 수직 필터로 걸러냄

In [ ]:

train_path = os.path.join(data_path, "train.json")
test_path = os.path.join(data_path, "test.json")
###################
#경로설정 오류

train = pd.read_json(train_path)
test = pd.read_json(test_path)


In [ ]:
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])

#np.array(data)
#X_band_1 array 선언

#np.array.astype(자료형)
#array 자료형 변환

X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate(
    [X_band_1[:, :, :, np.newaxis], 
     X_band_2[:, :, :, np.newaxis], 
     ((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]
     ], 
     axis=-1
)

#np.concatenate( , axis= )
#배열을 하나로 합침
#axis : 합치는 기준축
#axis=0 : 첫번째 요소 기준으로 붙이기 (행)
#axis=1 : 두번째 요소 기준으로 붙이기 (열)
######newaxis########

#axis=-1 : 마지막 축을 기준으로 붙임
# 75 * 75 * 3(axis=-1) 
# 3차원 축을 기준으로 붙여줌

#np.newaxis
#차원 늘려줌
#새로운 축 만들어줌

#H : horizontal
#V : vertical
#HH
#HV
#avg

#np.concatanete(a[:, :, np.newaxis], b[:, :, np.newaxis])
#2차원(평면) -> newaxis 기준으로 붙임 -> 3차원(입체)

#np.concatanete(a[:, :, :, np.newaxis], b[:, :, :, np.newaxis])
#3차원(입체) -> newaxis 기준으로 붙임 -> 4차원(입체)


In [ ]:
#Import Keras.
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

In [ ]:
#define model 
def getModel():
  #building model
  gmodel=Sequential()

#딥러닝 = 퍼셉트론 + 숨겨진 퍼셉트론 층 쌓음
#Sequential() : 퍼셉트론 층 만드는 함수
  
#Conv Layer 1
  gmodel.add(Conv2D(64, kernel_size=(3, 3), activation = 'relu', input_shape=(75, 75, 3)))

#gmodel.add() : 새로운 층을 추가하는 함수
#마지막 층(출력층) + 나머지 층(은닉층)

#CNN Convolution Neuron Network 
#입력된 이미지에 커널을 적용해서 특징을 한번 더 추출해낸다

#커널의 각 칸에 가중치가 있음
#따라서 기존 이미지에 가중치 곱해줌
#커널을 적용해서 만들어진 층 = Convolution

#Conv2D(커널 수, 커널 사이즈, 활성화함수, 입력값(행, 열, 색상=3/흑백=1))
#Convolution 층 추가하는 함수

  gmodel.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
  gmodel.app(Dropout(0.2))

#Pooling : 복잡한 내용 차원 축소 
#MaxPooling : 정해진 구역에서 최댓값 추출 -> 차원 축소
#AveragePooling : 정해진 구역에서 평균값 추출 -> 차원 축소

#MaxPooling2D(pool_size(풀 창 크기), strides())
#Dropout : 노드의 일부를 끔으로써 과적합 방지 (노드나 층이 너무 많으면 과적합 발생)

   #Conv Layer 2
  gmodel.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
  gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
  gmodel.add(Dropout(0.2))

    #Conv Layer 3
  gmodel.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
  gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
  gmodel.add(Dropout(0.2))

    #Conv Layer 4
  gmodel.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
  gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
  gmodel.add(Dropout(0.2))

    #Flatten the data for upcoming dense layers
  gmodel.add(Flatten())

#Flatten() : 2차원 배열 -> 1차원 배열
#########################
#1차원으로 바꿔야 활성화 함수가 있는 층에서 사용 가능
#Dense 1차원 배열이라 2차원이랑 곱할 수가 없음
  
    #Dense Layers
  gmodel.add(Dense(512))    #은닉층 Dense   #노드수 512
  gmodel.add(Activation('relu'))
  gmodel.add(Dropout(0.2))

    #Dense Layer 2
  gmodel.add(Dense(256))
  gmodel.add(Activation('relu'))
  gmodel.add(Dropout(0.2))

    #Sigmoid Layer  #출력층
  gmodel.add(Dense(1))
  gmodel.add(Activation('sigmoid'))
  
  mypotim=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#경사하강법 : 가중치 업데이트 
#고급경사하강법 : 모든 데이터를 다루기에 계산량이 많은 경사하강법의 단점 보완

#[고급경사하강법1] 확률적 경사하강법 : 랜덤 추출된 데이터 사용하여 가중치 업데이트
#[고급경사하강법2] 모멘텀 : 이전 업데이트와 같은 방향으로 일정 비율만 수정 -> 앞의 수정과 차이 줄임 -> 관성 유지
#[고급경사하강법3] 아담 : Momentum(정확도) + RMSProp(보폭크기) 개선 -> 가장 많이 쓰이는 경사하강법

  gmodel.compile(
      loss='binary_crossentropy', 
      optimizer=mypotim,
      metrics=['accuracy']
  )

#compile() : 모델이 구현될 환경 설정
#loss : 오차함수 선택

#loss = 'binary_crossentropy' (이항교차엔트로피)
#분류문제(예측값=참(1) | 거짓(0)) 에서 사용 

#교차 엔트로피 계열 함수
#오차함수 = 평균 제곱 오차 계열 + 교차 엔트로피 계열
#평균 제곱 오차 계열 = 수렴하기까지 속도 많이 걸림
#교차 엔트로피 계열 = 출력값에 로그 취함 -> 오차up 속도up 오차down 속도down 

#######################
#metrics() : 모델을 평가하는 지표 설정 + 평가 결과 보여줌 
#metics['accuracy'] : 모델을 정확도로 평가 + 정확도 보여줌

  gmodel.summary()
  return gmodel 

def get_callbacks(filepath, patience=2):
  es = EarlyStopping('val_loss', patience=patience, mode="min") 
  msave = ModelCheckpoint(filepath, save_best_only=True)
  return [es, msave]

#callback : loss 더 이상 줄지 않을 때 학습 종료

#epoch : 반복 횟수
#EarlyStopping(모니터링할 값, 기다릴 횟수, 모니터링할 값 최소화시켜야함) 
#val_loss : validation set의 loss
#성능이 증가하지 않는다고(테스트 오차가 줄지 않음) 판단 후 patience 횟수만큼 기다렸다 학습 중단
#############class 공부 필요################
#EarlyStopping = class -> 멈추는 것에 대한 함수 불러옴

#ModelCheckpoint(저장경로, 앞의 모델모다 나아졌을때만 저장하라)
#모니터할 값 val_loss(테스트 오차) 반환

file_path = ".model_weights.hdf5" #hdf5 #학습결과에 대한 모든 값 저장
callbacks = get_callbacks(filepath=file_path, patience=5) #######patience 재설정?


In [ ]:
target_train=train['is_iceberg']
X_train, X_valid, y_train, y_valid = train_test_split(X_train, target_train, random_state=1, train_size=0.8)

#train set -> train set + valid set
#바로 test하지 않고  valid set으로 과적합 판단 후 test

#train_test_split(data, target, random_state, train_size)
#train_size : 훈련 셋 구성 비율
#######random_state : 세트를 섞을 때  해당 값 보고 섞음, 하이퍼파라미터 튜닝 시 이 값을 고정하고 튜닝해야 데이터셋 변경 방지 가능



In [ ]:
#모델 실행 및 저장
import os
gmodel=getModel()
gmodel.fit(X_train_cv, y_train_cv, 
           batch_size=24, 
           epochs=50, 
           verbose=1, 
           validation_data=(X_valid, y_valid),
           callbacks=callbacks
)

#det getModel()
#batch_size : 한번에 입력될 데이터 수 
#verbose =1 : 학습 진행되는 과정 보여줌




In [ ]:
#저장된 weight 불러와서 loss, accuracy 확인
gmodel.load_weights(filepath=file_path) #hdf5 #학습한 데이터에 대한 모든 정보 담고 있음 #학습한 가중치를 불러옴
score=gmodel.evaluate(X_valid, y_valid, verbose=1)
print('Test loss:', score[0])
print('Test Accuracy:', score[1])

In [ ]:
predicted_test = gmodel.predict_proba(X_test)

In [ ]:
#제출파일 만들기
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=predicted_test.reshape((predicted_test.shape[0]))
submission.to_csv('sub.csv', index=False)